# GDAL Over the Web

Did you know that GDAL commands can be run directly against EOSDIS data over the Internet?

The [Geospatial Data Abstraction Library](https://gdal.org/) (GDAL) is a core tool for translating and processing geospatial data. This brief talk will explain how to configure GDAL's `vsicurl` driver to work with Earthdata Login, then demonstrate a few example use cases:

- Inspect metadata via gdalinfo
- Extract a layer or band via gdal_translate
- Subset to an area of interest via gdal_translate

Let's start with some setup and check what version of gdal we have installed.  Using commands against URLs protected by Earthdata Login requires at least GDAL v2.4.0, released Dec 2018.

In [ ]:
from IPython.display import Image

!gdalinfo --version

Typically, GDAL commands are run against files on your local file system.  Here we download a Landsat 8 product from the Landsat on AWS archive, which is publicly available on the web.  Then we run a [gdalinfo](https://gdal.org/programs/gdalinfo.html) command to see some basic information about the file.

In [ ]:
!wget http://landsat-pds.s3.amazonaws.com/c1/L8/217/072/LC08_L1TP_217072_20190107_20190130_01_T1/LC08_L1TP_217072_20190107_20190130_01_T1_BQA.TIF
!gdalinfo LC08_L1TP_217072_20190107_20190130_01_T1_BQA.TIF

GDAL also supports [virtual file systems](https://gdal.org/user/virtual_file_systems.html) for accessing data in memory, in compressed archives, or over a network as if they were local files.

Specifically, the [vsicurl](https://gdal.org/user/virtual_file_systems.html#vsicurl-http-https-ftp-files-random-access) driver allow GDAL commands to be run directly against files hosted on the Internet, using the syntax `<gdal command> /vsicurl/<url>`

Let's query that Landsat 8 scene again.

In [ ]:
!gdalinfo /vsicurl/http://landsat-pds.s3.amazonaws.com/c1/L8/217/072/LC08_L1TP_217072_20190107_20190130_01_T1/LC08_L1TP_217072_20190107_20190130_01_T1_BQA.TIF

That's great!  Most EOSDIS data is available over the Internet!  Let's try the same command against the download URL of a Sentinel-1 product hosted by ASF!

Unfortunately, this will fail with a cryptic "not recognized as a supported file format" error.

In [ ]:
!gdalinfo /vsicurl/https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-087-tops-20161117_20161030-161601-20645N_19970N-PP-1c8b-v2_0_1.nc

The command fails because this URL is protected by Earthdata Login.  To access data behind Earthdata Login, your client needs to do three things:

- Provide your username and password via a Basic Authentication header
- Follow redirects
- Support cookies

vsicurl supports all of these as of GDAL v2.4.0, released Dec 2018:

- .netrc support (enabled by default)
- Automatic following of redirects (enabled by default)
- Support for cookies by setting the GDAL_HTTP_COOKIEFILE and/or GDAL_HTTP_COOKIEJAR environment variables (disabled by default)

Let's set those missing environment variables and try again (assuming your Earthdata Login credentials are already in your .netrc file.)

In [ ]:
%env GDAL_HTTP_COOKIEJAR=cookies.txt
%env GDAL_HTTP_COOKIEFILE=cookies.txt

!gdalinfo /vsicurl/https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-087-tops-20161117_20161030-161601-20645N_19970N-PP-1c8b-v2_0_1.nc

This unlocks the full suite of GDAL commands for use directly against EOSDIS data on the web.  Don't like that this is a NETCDF product?  Try using [gdal_translate](https://gdal.org/programs/gdal_translate.html) to extract a single data layer as a .png image.

In [ ]:
!gdal_translate -of png \
                NETCDF:"/vsicurl/https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-087-tops-20161117_20161030-161601-20645N_19970N-PP-1c8b-v2_0_1.nc":/science/grids/data/amplitude \
                amplitude.png

Image(filename='amplitude.png')

Don't care about most of the image?  Try the `-projwin` option to crop to your particular area of interest.

In [ ]:
!gdal_translate -projwin_srs WGS84 \
                -projwin -156.72 20.62 -156.5 20.47 \
                -of png \
                NETCDF:"/vsicurl/https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-087-tops-20161117_20161030-161601-20645N_19970N-PP-1c8b-v2_0_1.nc":/science/grids/data/amplitude \
                cropped.png

Image(filename='cropped.png')

All without downloading the original product!

__Whether vsicurl actually saves you time over downloading the original product will vary depending on what GDAL command you're using, how well your data format supports byte range requests, and what vsicurl config options you set.__

vsicurl has many configuration options:
- The environment variables listed at https://trac.osgeo.org/gdal/wiki/ConfigOptions#GDALOGRHTTPoptions
- Any of the GDAL_HTTP_* options listed at https://gdal.org/api/cpl.html#_CPPv412CPLHTTPFetchPKc12CSLConstList

In particular, see CPL_CURL_VERBOSE and CPL_VSIL_CURL_CHUNK_SIZE.